# Edge Cloud Joint Inference with Seldon Core and Tempo

Description

## Setup Environment

In [ ]:
!conda env create --file ./conda/edge-cloud-inference.yaml

In [ ]:

!conda activate edge-cloud-inference

## Train models 

In [1]:
import os
from tempo.utils import logger
import logging
import numpy as np
logger.setLevel(logging.ERROR)
logging.basicConfig(level=logging.ERROR)
ARTIFACTS_FOLDER = os.getcwd()+"/artifacts"